In [0]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpej0p7n3h/pubring.gpg' created
gpg: /tmp/tmpej0p7n3h/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 19706 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.6.21-0ubuntu2_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.6.21-0ubuntu2) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.6.2

In [0]:
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive

In [0]:
#!mkdir -p my_drive/ML_Project
#!ls -p my_drive/ML_Project

In [0]:
import numpy
import pandas
import os
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.pipeline import Pipeline
from scipy import stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV 

In [4]:
!ls -p my_drive/ML_Project/X_train.txt

my_drive/ML_Project/X_train.txt


In [0]:
base = 'my_drive'+os.sep+'ML_Project'+os.sep

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
X = pandas.read_csv(base+"X_train.txt", delim_whitespace=True, header=None)
Y = pandas.read_csv(base+"Y_train.txt", delim_whitespace=True, header=None)

In [0]:
X = X.sample(n=10000,replace="False")
Y = Y.sample(n=10000,replace="False")

In [0]:
X = X.values
Y = Y.values

In [0]:
lbl = MinMaxScaler(copy=True, feature_range=(0, 1))
Y = lbl.fit_transform(Y)

In [0]:
#Y.columns.values[0] = '14'
#df_new = pandas.concat([X, Y],axis=1)
#df_new.drop_duplicates(inplace=True)
#df_new = df_new[(numpy.abs(stats.zscore(df_new)) < 3).all(axis=1)]

#X = df_new[list(df_new.columns[0:14])]
#Y = df_new.iloc[:,-1]

In [0]:
#X.drop(X.columns[[5, 6, 12]], axis=1, inplace=True)

In [14]:
X.shape

(100000, 14)

In [0]:
# evaluate model with standardized dataset
rf_estimators = []
rf_estimators.append(('standardize', MinMaxScaler(copy=True, feature_range=(0, 1))))
rf_estimator = RandomForestRegressor()
rf_estimators.append(('rf', RandomForestRegressor(random_state=seed)))
rf_pipeline = Pipeline(rf_estimators)

In [0]:
gb_estimators = []
gb_estimators.append(('standardize', MinMaxScaler(copy=True, feature_range=(0, 1))))
gb_estimator = GradientBoostingRegressor(random_state=seed)
gb_estimators.append(('gb', gb_estimator))
gb_pipeline = Pipeline(gb_estimators)

In [31]:
gb_pipeline.get_params().keys()

['standardize__copy',
 'gb__min_impurity_split',
 'gb__max_features',
 'gb__warm_start',
 'gb__min_samples_split',
 'gb__loss',
 'gb',
 'gb__alpha',
 'gb__learning_rate',
 'gb__verbose',
 'gb__presort',
 'gb__max_depth',
 'standardize__with_mean',
 'gb__criterion',
 'gb__min_impurity_decrease',
 'memory',
 'standardize',
 'gb__n_estimators',
 'gb__min_weight_fraction_leaf',
 'gb__random_state',
 'gb__max_leaf_nodes',
 'standardize__with_std',
 'gb__init',
 'gb__min_samples_leaf',
 'steps',
 'gb__subsample']

In [0]:
#kfold = KFold(n_splits=10, random_state=seed)
#results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
#print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [0]:
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
param_grid = {
    'rf__n_jobs':[8],
    'rf__max_depth': [15],
    'rf__max_features': [3],
    'rf__min_samples_leaf': [3],
    'rf__min_samples_split' : [3],
    'rf__n_estimators': [30, 100, 500, 1000]
}
grid_rf = GridSearchCV(estimator=rf_pipeline, cv=5, param_grid=param_grid, verbose=2)
grid_result_rf_2 = grid_rf.fit(X, Y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 
[CV]  rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 -  12.7s
[CV] rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.7s remaining:    0.0s


[CV]  rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 -  12.9s
[CV] rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 
[CV]  rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 -  12.9s
[CV] rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 
[CV]  rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 -  12.5s
[CV] rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 
[CV]  rf__max_depth=15, rf__n_estimators=30, rf__max_features=3, rf__min_samples_split=3, rf__min_samples_leaf=3, rf__n_jobs=8 -  13.1s
[CV] rf__max_depth=15, rf__n_estimators=100, rf__max_features=3, rf__min_sa

In [32]:
param_grid = {
  'gb__max_features': [3],
  'gb__min_samples_leaf': [3],
  'gb__min_samples_split': [3],
  'gb__n_estimators': [500],
  'gb__max_depth': [20],
	'gb__learning_rate': [0.05,0.1,0.15,0.2],
#	'gb__max_leaf_nodes': [3, 4, 5],
}
grid_gb = GridSearchCV(estimator=gb_pipeline, cv=5, param_grid=param_grid, verbose=2)
grid_result_gb = grid_gb.fit(X, Y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV]  gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -   1.6s
[CV] gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -   1.6s
[CV] gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -   1.6s
[CV] gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -   1.6s
[CV] gb__learning_rate=0.05, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__lea

[CV]  gb__learning_rate=0.1, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=5 -   1.8s
[CV] gb__learning_rate=0.1, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=5 
[CV]  gb__learning_rate=0.1, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=5 -   1.9s
[CV] gb__learning_rate=0.1, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=5 
[CV]  gb__learning_rate=0.1, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=5 -   1.9s
[CV] gb__learning_rate=0.1, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=5 
[CV]  gb__learning_

[CV]  gb__learning_rate=0.2, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -   1.5s
[CV] gb__learning_rate=0.2, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=4 
[CV]  gb__learning_rate=0.2, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=4 -   1.7s
[CV] gb__learning_rate=0.2, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=4 
[CV]  gb__learning_rate=0.2, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=4 -   1.8s
[CV] gb__learning_rate=0.2, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=4 
[CV]  gb__learning_

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.9min finished


In [40]:
param_grid = {
  'gb__max_features': [3],
  'gb__min_samples_leaf': [3],
  'gb__min_samples_split': [3],
  'gb__n_estimators': [500],
  'gb__max_depth': [20],
	'gb__learning_rate': [0.01],
	'gb__max_leaf_nodes': [3],
}
grid_gb = GridSearchCV(estimator=gb_pipeline, cv=5, param_grid=param_grid, verbose=2)
grid_result_gb = grid_gb.fit(X, Y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -  52.4s
[CV] gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   52.4s remaining:    0.0s


[CV]  gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -  46.9s
[CV] gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 - 1.1min
[CV] gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 -  51.1s
[CV] gb__learning_rate=0.01, gb__n_estimators=500, gb__max_depth=20, gb__max_features=3, gb__min_samples_split=3, gb__min_samples_leaf=3, gb__max_leaf_nodes=3 
[CV]  gb__lea

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.6min finished


In [41]:
# summarize results
def summarize(grid_result,method):
  print (method)
  print ('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
#summarize(grid_result_rf,'random forest sample')
#print('*'*80)
summarize(grid_result_rf_2,'random forest full set')
summarize(grid_result_gb,'gradient boosting')

random forest full set
Best: -0.006486 using {'rf__max_depth': 20, 'rf__n_estimators': 500, 'rf__max_features': 3, 'rf__min_samples_split': 3, 'rf__min_samples_leaf': 16, 'rf__n_jobs': 8}
gradient boosting
Best: 0.048272 using {'gb__learning_rate': 0.01, 'gb__n_estimators': 500, 'gb__max_depth': 20, 'gb__max_features': 3, 'gb__min_samples_split': 3, 'gb__min_samples_leaf': 3, 'gb__max_leaf_nodes': 3}


In [0]:
X_test = pandas.read_csv(base+"X_test.txt", delim_whitespace=True, header=None)

In [0]:
#X_test.drop(X_test.columns[[5, 6, 12]], axis=1, inplace=True)

In [0]:
# evaluate model with standardized dataset
X_test = X_test.values
prediction = grid_result_gb.predict(X_test)

In [0]:
fh = open(base+'predictions_gb.csv','w')
fh.write('ID,Prediction\n')
for i,yi in enumerate(prediction):
    fh.write('{},{}\n'.format(i,yi))
fh.close()

In [46]:
unique, counts = numpy.unique(prediction, return_counts=True)
print (numpy.asarray((unique, counts)).T)

[[ 0.06158188  1.        ]
 [ 0.08455057 19.        ]
 [ 0.09481692  7.        ]
 ...
 [ 4.64751544  1.        ]
 [ 4.74031432  1.        ]
 [ 4.99573631  1.        ]]
